In [1]:
import numpy as np

In [2]:
def identity(x):
    return x
def d_identity_dx(x):
    return np.ones(shape=x.shape, like=x)

In [3]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def d_sigmoid_dx(x):
    return sigmoid(x)*(1-sigmoid(x))
sigmoid(-1000), sigmoid(0), sigmoid(1000)

/var/folders/hd/kdltsywn53jbp7lv3g1hld3w0000gn/T/ipykernel_52479/3566510964.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


(0.0, 0.5, 1.0)

In [45]:
class DenseLayer:
    
    def __init__(self, activation_function, dactivation_function):
        self.activation_function=activation_function
        self.dactivation_function=dactivation_function
    
    def foward_pass(self, x):
        x = np.array(x)
        # print(self.w.shape, x.shape)
        s = np.matmul(self.w, x)
        # print(s)
        return self.activation_function(s)
    
    def backward_pass(self, x):
        x = np.array(x)
        
        # print('w.shape', self.w.shape)
        
        # print('x.shape', x.shape)
        
        s = np.matmul(self.w, x)
        
        # print('s.shape', s.shape)
        
        dactivation = np.array(self.dactivation_function(s))
        
        # print('dactivation.shape', dactivation.shape)
        
        # d/dx
        dsdx = self.w.T
        
        dx = np.matmul(dsdx, dactivation)
        
        # d/dw
        dsdw = x
        
        dw = np.array([ x*dsdw for x in dactivation ])
        
        # print('dw', dw)
        
        return dx, dw
    

In [46]:

z1 = DenseLayer(sigmoid, d_sigmoid_dx)
z2 = DenseLayer(sigmoid, d_sigmoid_dx)
y =  DenseLayer(identity, d_identity_dx)

z1.w = np.array(
    [
        [-1, -2, -3],
        [ 1,  2,  3]
    ]
)
z2.w = np.array(
    [
        [-1, -2, -3],
        [ 1,  2,  3]
    ]
)
y.w = np.array(
    [
        [ -1,  2, -1.5]
    ]
)


In [47]:
def model(x):
    Z1 = [1, *z1.foward_pass(x)]
    Z2 = [1, *z2.foward_pass(Z1)]
    Y =  y.foward_pass(Z2)
    return Y

In [48]:
model(np.array([1,1,1]))

array([-2.43689523])

In [49]:
def compute_gradient_via_backprop(x, y_true):
    Z1 = np.array([1, *z1.foward_pass(x)])
    Z2 = np.array([1, *z2.foward_pass(Z1)])
    Y =  y.foward_pass(Z2)
    print(Y)
    L = (1/2) * (Y-y_true)**2
    
    dLdy = Y-y_true
    
    dydz2, dydw3 = y.backward_pass(Z2)
    
    # print('dydz2\n', dydz2)
    
    dLdw3 = dLdy*dydw3
    
    print('dLdw3', dLdw3)
    
    dz2dz1, dz2dw2 = z2.backward_pass(Z1)
    
    # print('dz2dz1\n', dz2dz1)
    
    dLdw2 = dLdy * dydz2 * dz2dw2
    
    print('dLdw2', dLdw2)
    
    dz1dx, dz1dw1 = z1.backward_pass(x)
    
    # print('dz1dx\n', dz1dx)
    
    dLdw1 = (dLdy * dydz2 * dz2dz1).T * dz1dw1 # why did this need that transpose?
    
    print('dLdw1', dLdw1)
    
compute_gradient_via_backprop(np.array([1,1,1]), np.array(1))

[-2.43689523]
dLdw3 [[-3.43689523 -0.061967   -3.37492823]]
dLdw2 [[ 0.06084974 -0.00030092  0.09104892]
 [ 0.06084974 -0.00030092  0.09104892]]
dLdw1 [[-7.94096102e-19  3.17638441e-18 -3.61754891e-18]
 [-7.94096102e-19  3.17638441e-18 -3.61754891e-18]]
